In [ ]:
# Does not need to be executed if ~/.ipython/profile_default/ipython_config.py
# exists and contains get_config().InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

Let $n$ be an integer at least equal to 1. A total of $n$ disks all of different sizes, stacked at position 0, have to be moved to position 2, with position 1 available, thanks to a succession of moves each of which consists of removing the disk at the top of a stack and bringing it to the top of another (possibly empty) stack. At any stage of the game, no disk should be above a smaller disk; in particular, at the beginning of the game, all disks are stacked from largest to smallest.

It is necessary to eventually move the largest disk from position 0 to position 2. That is possible only when the $n-1$ smaller disks have been moved away from position 0 and stacked at position 1. Then, with the largest disk having found its final position at the base of position 2, those disks have to be moved away from position 1 and stacked at position 2. This is illustrated as follows for $n=4$.

Initial configuration:

![](stage_1.pdf)

After the $n-1$ smaller disks have been moved from position 0 to position 1:

![](stage_2.pdf)

Largest disk moved from position 0 to position 2:

![](stage_3.pdf)

Final configuration, after the $n-1$ smaller disks have been moved from position 1 to position 2:

![](stage_4.pdf)

The recursive implementation immediately follows from that observation:

In [ ]:
def recursive__towers(n, start_pos, end_pos, intermediate_pos):
    if n == 1:
        print(f'Move smallest disk from position {start_pos} '
              f'to position {end_pos}'
             )
    else:
        recursive__towers(n - 1, start_pos, intermediate_pos, end_pos)
        print(f'Move disk of size {n} from position {start_pos} '
              f'to position {end_pos}'
             )
        recursive__towers(n - 1, intermediate_pos, end_pos, start_pos)

recursive__towers(4, 0, 2, 1)

Also, that observation establishes that the output of the recursive implementation is not only a solution; it is in fact the only possible solution for that optimal number of moves, equal to $2^n-1$, as shown by induction on $n$:

* If $n=1$, then $2^n-1=1$, and 1 move is necessary and sufficient indeed.
* If $2^n-1$ moves are necessary and sufficient to move $n$ disks, then $(2^n-1)\times 2 + 1= 2^{n+1}-1$ moves are necessary and sufficient indeed to move $n+1$ disks.

Moving the smaller $n-1$ disks, then moving the largest disk, then moving the $n-1$ smaller disks again to move the $n$ disks, done recursively, shows that the Hanoi of towers puzzle is equivalent to that of putting ticks of $n$ different sizes on a ruler, as illustrated next for $n=4$:

![](ruler.pdf)

This shows that every second move involves moving the smallest disk; the other moves are clearly imposed. So to convert the recursive implementation into an iterative implementation, it suffices to determine where to move the smallest disk every time it has to be moved.

Place the three positions, 0, 1 and 2, on a ring.

![](cycle.pdf)

* Moving the $n$ disks from position 0 to position 2 can be visualised as moving on the ring counterclockwise.
* Moving the $n-1$ disks from position 0 to position 1 can be visualised as moving on the ring clockwise.
* Moving the $n-1$ disks from position 1 to position 2 can be visualised as moving on the ring clockwise.

It follows that:

* the stack of $n$ disks have to be moved from where they are to the next position counterclockwise, which if $n>1$ involves
* twice moving the stack of the $n-1$ smaller disks from where they are to the next position clockwise, which if $n>2$ involves
* four times moving the stack of the $n-2$ smaller disks from where they are to the next position counterclockwise, which if $n>3$ involves
* eight times moving the stack of the $n-3$ smaller disks from where they are to the next position clockwise
* ...

Hence:

* if $n$ is even then the smallest disk always has to be moved to the next position clockwise, that is, from 0 to 1, from 1 to 2, and from 2 to 0;
* if $n$ is odd then the smallest disk always has to be moved to the next position counterclockwise, that is, from 0 to 2, from 2 to 1, and from 1 to 0.

Note that the expression `1 - n % 2 * 2` evalues to 1 if `n` is even, and to -1 if `n` is odd:

In [ ]:
n = 2; 1 - n % 2 * 2
n = 3; 1 - n % 2 * 2

So `1 - n % 2 * 2` just needs to be added to the currrent position of the smallest disk to, modulo 3, give the next position of the smallest disk.

We can use a tuple of 3 lists, `stacks`, to represent the stacks of disks at position 0, 1 and 2, respectively. We can use the integers between 1 and $n$ to denote the $n$ disks, from smallest to largest. To start with, all disks are on the first stack, stacked up from largest to smallest:

In [ ]:
n = 4
stacks = list(range(n, 0, -1)), [], []

stacks

The smallest disk is at position 0. With $n$ set to 4, it will have to move clockwise, so to position 1. A combination of `pop()` and `append()` realises the move:

In [ ]:
small_disk_pos = 0
direction = 1 - n % 2 * 2
new_small_disk_pos = (small_disk_pos + direction) % 3

print(f'Move smallest disk to position {new_small_disk_pos}')
stacks[new_small_disk_pos].append(stacks[small_disk_pos].pop())

stacks

When not moving the smallest disk, one has to work with the two positions different to the new position of the smallest disk:

In [ ]:
small_disk_pos = new_small_disk_pos

small_disk_pos
(small_disk_pos + 1) % 3
(small_disk_pos + 2) % 3

If there is an empty stack at one of these positions, then that position should be the destination position for the next move. Otherwise, if the stacks at both positions are not empty, then the position whose stack has the largest disk at the top should be the destination position. So to determine the destination position, one can sort both positions using a key which can be $n+1$ for a position where the stack is empty, and for a position where the stack $S$ is not empty, the integer between 1 and n that represents the size of the disk at the top of $S$; that way, the position sorted second (and last) is guaranteed to be the destination position. The second move is again realised by a combination of `pop()` and `append()`:

In [ ]:
stacks

from_pos, to_pos =\
      sorted(((small_disk_pos + 1) % 3,
              (small_disk_pos + 2) % 3
             ), key = lambda x: not stacks[x] and n + 1 or\
                                stacks[x][-1]
            )
print(f'Move disk from position {from_pos} to position {to_pos}')
stacks[to_pos].append(stacks[from_pos].pop())

stacks

It suffices to perform those operations again and again. Illustrating doing them twice again:

In [ ]:
stacks

new_small_disk_pos = (small_disk_pos + direction) % 3
print(f'Move smallest disk to position {new_small_disk_pos}')
stacks[new_small_disk_pos].append(stacks[small_disk_pos].pop())

stacks

small_disk_pos = new_small_disk_pos
from_pos, to_pos =\
      sorted(((small_disk_pos + 1) % 3,
              (small_disk_pos + 2) % 3
             ), key = lambda x: not stacks[x] and n + 1 or\
                                stacks[x][-1]
            )
print(f'Move disk from position {from_pos} to position {to_pos}')
stacks[to_pos].append(stacks[from_pos].pop())

stacks

In [ ]:
stacks

new_small_disk_pos = (small_disk_pos + direction) % 3
print(f'Move smallest disk to position {new_small_disk_pos}')
stacks[new_small_disk_pos].append(stacks[small_disk_pos].pop())

stacks

small_disk_pos = new_small_disk_pos
from_pos, to_pos =\
      sorted(((small_disk_pos + 1) % 3,
              (small_disk_pos + 2) % 3
             ), key = lambda x: not stacks[x] and n + 1 or\
                                stacks[x][-1]
            )
print(f'Move disk from position {from_pos} to position {to_pos}')
stacks[to_pos].append(stacks[from_pos].pop())

stacks

Putting it all together:

In [ ]:
def iterative_towers(n, start_pos, end_pos, intermediate_pos):
    small_disk_pos = 0
    direction = 1 - n % 2 * 2
    stacks = list(range(n, 0, -1)), [], []
    for i in range(2 ** n - 1):
        if i % 2 == 0:
            new_small_disk_pos = (small_disk_pos + direction) % 3
            print(f'Move smallest disk from position {small_disk_pos} '
                  f'to position {new_small_disk_pos}'
                 )
            stacks[new_small_disk_pos].append(stacks[small_disk_pos].pop())
            small_disk_pos = new_small_disk_pos
        else:
            from_pos, to_pos =\
                  sorted(((small_disk_pos + 1) % 3,
                          (small_disk_pos + 2) % 3
                         ), key = lambda x: not stacks[x] and n + 1 or\
                                            stacks[x][-1]
                        )
            stacks[to_pos].append(stacks[from_pos].pop())
            print(f'Move disk of size {stacks[to_pos][-1]} '
                  f'from position {from_pos} to position {to_pos}'
                 )

            
iterative_towers(4, 0, 2, 1)